In [2]:
import sys
!{sys.executable} -m pip install scikit-image

  Using cached scikit_image-0.18.1-cp38-cp38-win_amd64.whl (12.2 MB)
  Using cached tifffile-2021.3.17-py3-none-any.whl (163 kB)
  Using cached imageio-2.9.0-py3-none-any.whl (3.3 MB)
  Using cached networkx-2.5-py3-none-any.whl (1.6 MB)
  Using cached PyWavelets-1.1.1-cp38-cp38-win_amd64.whl (4.3 MB)


You should consider upgrading via the 'c:\users\monis\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [5]:
!{sys.executable} -m pip install torch

  Using cached torch-1.8.1-cp38-cp38-win_amd64.whl (190.5 MB)


You should consider upgrading via the 'c:\users\monis\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [6]:
from PIL import Image
import numpy as np
from skimage import color
import torch
import torch.nn.functional as F
from IPython import embed

In [7]:
def load_img(img_path):
    out_np = np.asarray(Image.open(img_path))
    if(out_np.ndim==2):
        out_np = np.tile(out_np[:,:,None],3)
    return out_np

In [8]:
def resize_img(img, HW=(256,256), resample=3):
    return np.asarray(Image.fromarray(img).resize((HW[1],HW[0]), resample=resample))

In [10]:
def preprocess_img(img_rgb_orig, HW=(256,256), resample=3):
    # return original size L and resized L as torch Tensors
    img_rgb_rs = resize_img(img_rgb_orig, HW=HW, resample=resample)
    
    img_lab_orig = color.rgb2lab(img_rgb_orig)
    img_lab_rs = color.rgb2lab(img_rgb_rs)

    img_l_orig = img_lab_orig[:,:,0]
    img_l_rs = img_lab_rs[:,:,0]

    tens_orig_l = torch.Tensor(img_l_orig)[None,None,:,:]
    tens_rs_l = torch.Tensor(img_l_rs)[None,None,:,:]

    return (tens_orig_l, tens_rs_l)

In [11]:
def postprocess_tens(tens_orig_l, out_ab, mode='bilinear'):
    # tens_orig_l 1 x 1 x H_orig x W_orig
    # out_ab 1 x 2 x H x W

    HW_orig = tens_orig_l.shape[2:]
    HW = out_ab.shape[2:]

    # call resize function if needed
    if(HW_orig[0]!=HW[0] or HW_orig[1]!=HW[1]):
        out_ab_orig = F.interpolate(out_ab, size=HW_orig, mode='bilinear')
    else:
        out_ab_orig = out_ab

    out_lab_orig = torch.cat((tens_orig_l, out_ab_orig), dim=1)
    return color.lab2rgb(out_lab_orig.data.cpu().numpy()[0,...].transpose((1,2,0)))
